In [1]:
import ee
import geemap

# Authentifier et initialiser Google Earth Engine
ee.Authenticate()
ee.Initialize(project="devoir-sur-table")


Enter verification code: 4/1ASc3gC1l75idFFF0Yd6Cah2i2Uz7ULG7dliAbFyBCvYoATBYGQBslYuI2M8

Successfully saved authorization token.


In [2]:
import ee
ee.Initialize(project="devoir-sur-table")

print("Earth Engine est connecté :", ee.Number(1).getInfo())


Earth Engine est connecté : 1


In [3]:
# Charger la collection ERA5 Daily
era5_daily = ee.ImageCollection("ECMWF/ERA5/DAILY")

# Vérification
print("Nombre d'images dans ERA5 Daily :", era5_daily.size().getInfo())

# Affichage de la liste des bandes disponibles (sur la première image)
print("Bandes disponibles :",
      era5_daily.first().bandNames().getInfo())


Nombre d'images dans ERA5 Daily : 15165
Bandes disponibles : ['mean_2m_air_temperature', 'minimum_2m_air_temperature', 'maximum_2m_air_temperature', 'dewpoint_2m_temperature', 'total_precipitation', 'surface_pressure', 'mean_sea_level_pressure', 'u_component_of_wind_10m', 'v_component_of_wind_10m']


In [6]:
# ==============================================================================
# SÉLECTION DE LA BANDE total_precipitation (ERA5 DAILY)

# Sélection de la bande de précipitation totale
era5_precip = era5_daily.select("total_precipitation")

# Vérifications
print("Nombre d'images :", era5_precip.size().getInfo())
print("Bandes :", era5_precip.first().bandNames().getInfo())


Nombre d'images : 15165
Bandes : ['total_precipitation']


In [8]:
# ==============================================================================
# GÉOMÉTRIE DU GHANA À PARTIR DE FAO/GAUL/2015/level0
# ==============================================================================

# Collection des pays (niveau 0)
gaul0 = ee.FeatureCollection("FAO/GAUL/2015/level0")

# Sélection du Ghana (le nom est stocké dans "ADM0_NAME")
ghana_fc = gaul0.filter(ee.Filter.eq("ADM0_NAME", "Ghana"))

# Géométrie (polygone du pays)
ghana_geom = ghana_fc.geometry()

# Vérifications
print("Nombre d'entités Ghana :", ghana_fc.size().getInfo())
print("Type géométrie :", ghana_geom.type().getInfo())


Nombre d'entités Ghana : 1
Type géométrie : MultiPolygon


In [9]:
# Filtrage sur la période 2015-2020
era5_2015_2020 = (
    era5_daily
    .filterDate("2015-01-01", "2020-12-31")
)

# Vérification
print("Nombre d'images (2015–2020) :", era5_2015_2020.size().getInfo())
print("Première date :", era5_2015_2020.first().date().format("YYYY-MM-dd").getInfo())
print("Dernière date :", era5_2015_2020.sort("system:time_start", False)
      .first().date().format("YYYY-MM-dd").getInfo())


Nombre d'images (2015–2020) : 2017
Première date : 2015-01-01
Dernière date : 2020-07-09


In [10]:
# ==============================================================================
# PRÉCIPITATIONS JOURNALIÈRES -> PRÉCIPITATIONS ANNUELLES MOYENNES (2015–2020)
# ERA5 total_precipitation est en mètres (m) -> conversion en millimètres (mm)
# Sortie : une Image (mm/an) = moyenne des totaux annuels sur 2015-2020
# =============================================================================

# --- Ghana geometry (GAUL level0) ---
gaul0 = ee.FeatureCollection("FAO/GAUL/2015/level0")
ghana_geom = gaul0.filter(ee.Filter.eq("ADM0_NAME", "Ghana")).geometry()

# --- ERA5 Daily precipitation (m) filtered 2015-2020 ---
era5_precip = (ee.ImageCollection("ECMWF/ERA5/DAILY")
               .select("total_precipitation")
               .filterDate("2015-01-01", "2020-12-31")
               .map(lambda img: img.multiply(1000)  # m -> mm
                    .rename("tp_mm")
                    .copyProperties(img, ["system:time_start"]))
              )

# --- Annual totals (mm/year) ---
years = ee.List.sequence(2015, 2020)

def annual_total(y):
    y = ee.Number(y).toInt()
    start = ee.Date.fromYMD(y, 1, 1)
    end   = start.advance(1, "year")
    tot = (era5_precip.filterDate(start, end)
           .sum()
           .rename("annual_tp_mm")
           .set("year", y)
           .set("system:time_start", start.millis()))
    return tot

annual_totals = ee.ImageCollection(years.map(annual_total))

# --- Mean annual precipitation over 2015-2020 (mm/year) ---
mean_annual_precip = (annual_totals.mean()
                      .rename("mean_annual_tp_mm")
                      .clip(ghana_geom))

# Quick checks
print("Nb images annuelles :", annual_totals.size().getInfos())
print("Bandes (moyenne annuelle) :", mean_annual_precip.bandNames().getInfos())


Nb images annuelles : 6
Bandes (moyenne annuelle) : ['mean_annual_tp_mm']


In [15]:
import pandas as pd

# --- Dossier outputs ---
out_dir = r"C:\Users\admin\Pictures\DEVOIR\DOSSIER_EVALUATION\outputs"
os.makedirs(out_dir, exist_ok=True)

# --- Ghana geometry (GAUL level0) ---
gaul0 = ee.FeatureCollection("FAO/GAUL/2015/level0")
ghana_geom = gaul0.filter(ee.Filter.eq("ADM0_NAME", "Ghana")).geometry()

# --- Mean annual precipitation image (mm/year) over 2015-2020 ---
era5_precip = (ee.ImageCollection("ECMWF/ERA5/DAILY")
               .select("total_precipitation")
               .filterDate("2015-01-01", "2020-12-31")
               .map(lambda img: img.multiply(1000).rename("tp_mm")
                    .copyProperties(img, ["system:time_start"]))
              )

years = ee.List.sequence(2015, 2020)

def annual_total(y):
    y = ee.Number(y).toInt()
    start = ee.Date.fromYMD(y, 1, 1)
    end   = start.advance(1, "year")
    return (era5_precip.filterDate(start, end)
            .sum()
            .rename("annual_tp_mm")
            .set("year", y)
            .set("system:time_start", start.millis()))

annual_totals = ee.ImageCollection(years.map(annual_total))

mean_annual_precip = (annual_totals.mean()
                      .rename("mean_annual_tp_mm")
                      .clip(ghana_geom))

# --- Statistiques spatiales (Ghana)
scale_m = 25000

stats = (mean_annual_precip.reduceRegion(
            reducer=ee.Reducer.mean()
                    .combine(ee.Reducer.variance(), sharedInputs=True)
                    .combine(ee.Reducer.stdDev(), sharedInputs=True),
            geometry=ghana_geom,
            scale=scale_m,
            bestEffort=True,
            maxPixels=1e13
        ))

stats_dict = stats.getInfo()

# Récupération des valeur
mean_val = stats_dict.get("mean_annual_tp_mm_mean")
var_val  = stats_dict.get("mean_annual_tp_mm_variance")
sd_val   = stats_dict.get("mean_annual_tp_mm_stdDev")
cv_val   = (sd_val / mean_val) if (mean_val is not None and mean_val != 0) else None

# --- Enregistrement CSV ---
df_out = pd.DataFrame([{
    "country": "Ghana",
    "period_start": "2015-01-01",
    "period_end": "2020-12-31",
    "variable": "mean_annual_precipitation",
    "unit": "mm/year",
    "spatial_mean": mean_val,
    "spatial_variance": var_val,
    "spatial_stdDev": sd_val,
    "spatial_CV": cv_val
}])

csv_file = os.path.join(out_dir, "ghana_precip_annual_mean_spatial_stats_2015_2020.csv")
df_out.to_csv(csv_file, index=False)

# Affichage
print(df_out)
print("CSV enregistré :", csv_file)


  country period_start  period_end                   variable     unit  \
0   Ghana   2015-01-01  2020-12-31  mean_annual_precipitation  mm/year   

   spatial_mean  spatial_variance  spatial_stdDev  spatial_CV  
0   1048.937798      94587.221923      307.550357    0.293202  
CSV enregistré : C:\Users\admin\Pictures\DEVOIR\DOSSIER_EVALUATION\outputs\ghana_precip_annual_mean_spatial_stats_2015_2020.csv
